In [1]:
from bigbrotr import Bigbrotr
from event import Event
from relay import Relay
from relay_metadata import RelayMetadata

In [49]:
import sys
import json
import time
import websocket
import ssl
import socks  # Provided by PySocks
import socket

def fetch_nostr_events(url, network, start, stop):
    # Generate a random subscription ID
    events = []
    sub_id = "" + str(int(time.time()))

    # Filter to get all events between start and stop
    req = [
        "REQ",
        sub_id,
        {
            "since": int(start),
            "until": int(stop)
        }
    ]

    close_req = ["CLOSE", sub_id]

    # Configure proxy if using Tor
    if network.lower() == "tor":
        socks.set_default_proxy(socks.SOCKS5, "127.0.0.1", 9050)
        socket.socket = socks.socksocket
        sslopt = {"cert_reqs": ssl.CERT_NONE}
    else:
        sslopt = {"cert_reqs": ssl.CERT_REQUIRED}

    def on_message(ws, message):
        data = json.loads(message)
        if data[0] == "EVENT":
            try:
                Event.from_dict(data[2])
            except Exception as e:
                print(data[2])
            events.append(data[2])
        elif data[0] == "EOSE":
            print("End of stored events.")
            ws.send(json.dumps(close_req))
            ws.close()

    def on_error(ws, error):
        print("Error:", error)

    def on_close(ws, close_status_code, close_msg):
        print("Connection closed.")


    def on_open(ws):
        ws.send(json.dumps(req))

    print(f"Connecting to {url} over {network}...")
    ws = websocket.WebSocketApp(
        url,
        on_message=on_message,
        on_error=on_error,
        on_close=on_close,
        on_open=on_open
    )

    ws.run_forever(sslopt=sslopt)
    print("Connection closed.")
    print(f"Fetched {len(events)} events.")
    return events

# Example usage:
events = fetch_nostr_events("wss://google.com", "clearnet", time.time()-100, time.time())


Connecting to wss://google.com over clearnet...
Error: Handshake status 400 Bad Request -+-+- {'content-type': 'text/html; charset=UTF-8', 'referrer-policy': 'no-referrer', 'content-length': '1555', 'date': 'Sun, 04 May 2025 17:17:46 GMT', 'alt-svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000', 'connection': 'close'} -+-+- b'<!DOCTYPE html>\n<html lang=en>\n  <meta charset=utf-8>\n  <meta name=viewport content="initial-scale=1, minimum-scale=1, width=device-width">\n  <title>Error 400 (Bad Request)!!1</title>\n  <style>\n    *{margin:0;padding:0}html,code{font:15px/22px arial,sans-serif}html{background:#fff;color:#222;padding:15px}body{margin:7% auto 0;max-width:390px;min-height:180px;padding:30px 0 15px}* > body{background:url(//www.google.com/images/errors/robot.png) 100% 5px no-repeat;padding-right:205px}p{margin:11px 0 22px;overflow:hidden}ins{color:#777;text-decoration:none}a img{border:0}@media screen and (max-width:772px){body{background:none;margin-top:0;max-width:none;pad